### GSM8k

Example code for GSM8k

In [1]:
from agentquest.benchmarks.gsm8k import Gsm8kDriver, Gsm8kUtils, Gsm8kAction

/home/bgautam/ai_quest_project/agentquest/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# valid_categories = ["main", "socratic"]
# valid_splits = ["train", "test"]
dataset = Gsm8kUtils.load_data(category="main", split="train")

In [3]:
dataset

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...
...,...,...
7468,"Very early this morning, Elise left home in a ...","For the distance she traveled, Elise paid 23 -..."
7469,Josh is saving up for a box of cookies. To rai...,He makes $.5 profit on each bracelet because 1...
7470,Colin can skip at six times the speed that Bra...,Tony can skip at twice the speed that Bruce ca...
7471,"Janet, a third grade teacher, is picking up th...",Janet needs 35 lunches for the kids + 5 for th...


### Non-interactive mode
This is basically, the agent has one shot to give the answer.

In [4]:
accumulated_metrics = []
for problem in dataset.to_dict(orient="records"):
    print(problem)
    driver = Gsm8kDriver(
        problem=problem["question"], goal=problem["answer"], interactive=False
    )
    obs = driver.reset()
    print(obs.output)
    human_input = input(obs.output)  # Can be replaced by agent input
    # obs = driver.step(Gsm8kAction(value=str(human_input)))
    obs = driver.step_raw(str(human_input))
    print(obs.output)
    accumulated_metrics.append(driver.metrics.export())

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?
Your response must in the following format 
Answer: <answer>

Correct answer!!!
2024-12-17 14:02 - WARNING - Exception encountered: Metrics.get_repetition_rate() missing 1 required positional argument: 'num_execution_steps'. Unable to calculate repetition rate.
2024-12-17 14:02 - WARNING - Exception encountered: . Unable to calculate progress rate.
{'question': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?', 'answer': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 p

KeyboardInterrupt: Interrupted by user

In [5]:
accumulated_metrics

[{'problem': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?',
  'goal': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72',
  'success': True,
  'actions': [{'value': '72'}],
  'states': [{'value': '72'}],
  'observations': [{'output': 'Correct answer!!!',
    'success': True,
    'can_proceed': False}]},
 {'problem': 'Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?',
  'goal': 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10',
  'success': True,
  'actions': [{'value': '10'}],
  'states': [{'value': '10'}],
  'observations': [{'output': 'Correct answer!!!',
    'success': True,
    'can_proceed': False}]},
 {'problem': 'Betty is saving money for a new wallet which

### Interactive mode. 
The agent has multiple shots for one problem.

In [ ]:
accumulated_metrics = []
for problem in dataset.to_dict(orient="records"):
    print(problem)
    driver = Gsm8kDriver(
        problem=problem["question"], goal=problem["answer"], interactive=True
    )
    obs = driver.reset()

    num_attempts = 1
    while not obs.success and obs.can_proceed and (num_attempts <= 5):
        human_input = input(obs.output)  # Can be replaced by agent input
        obs = driver.step(Gsm8kAction(value=str(human_input)))
        # obs = driver.step_raw(str(human_input))
        print(obs.output)
        num_attempts += 1

    accumulated_metrics.append(
        driver.metrics.export(
            repetition_function_kwargs={"theta_a": 1, "num_execution_steps": 5}
        )
    )

In [ ]:
accumulated_metrics